In [ ]:
!pip install dalex

     |████████████████████████████████| 427 kB 8.1 MB/s 
     |████████████████████████████████| 275 kB 43.4 MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.2.0-py3-none-any.whl size=433519 sha256=2e34aab1c666dce82597db758e330d691dec6cf8bdb05b05cba59821393208da
  Stored in directory: /root/.cache/pip/wheels/ef/70/33/efea781f1c5de2489d9da29b1cfd6053a855423ffbe904a3bc
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=491be4d2d75ee4b81314726eb7c0301a1bba6861c70c37dbb8ce8c531485e8dc
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built anchor-exp lime


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import pandas as pd
import json
import re
import ast
import dalex
import numpy as np

In [ ]:
def process_spectra(file_obj, file_name):
    file_string = str(file_obj[file_name])

    return file_string.split("\\n")

In [ ]:
def process_matrix(file_obj, file_name):
    file_string = str(file_obj[file_name])

    result = file_string.split("\\n")
    result[0] = result[0].replace("b'", "")

    for i in range(len(result)):
        result[i] = result[i].split(" ")
        result[i] = [int(x) if x.isdigit() else x for x in result[i]]
        lastel = result[i][-1]
        if lastel == "+":
            result[i][-1] = "PASS"
        elif lastel == "-":
            result[i][-1] = "FAIL"
    
    return result

In [ ]:
spectra_file = files.upload()
matrix_file = files.upload()

Saving spectra to spectra (1)


Saving matrix to matrix (1)


In [ ]:
spectra_list = process_spectra(spectra_file, list(spectra_file.keys())[0])
spectra_list[-1] = "Pass/Fail"
spectra_list[0] = spectra_list[0].replace("b'", "")

In [ ]:
matrix_list = process_matrix(matrix_file, list(matrix_file.keys())[0])

In [ ]:
matrix_list.pop()

["'"]

In [ ]:
df = pd.DataFrame(data=matrix_list, columns=spectra_list)

In [ ]:
df

,org.apache.commons.math3.util.MathArrays$1#674,org.apache.commons.math3.util.MathArrays$1#678,org.apache.commons.math3.util.MathArrays$1#680,org.apache.commons.math3.util.MathArrays$1#681,org.apache.commons.math3.util.MathArrays$1#683,org.apache.commons.math3.util.MathArrays$1#684,org.apache.commons.math3.util.MathArrays$1#687,org.apache.commons.math3.util.MathArrays$1#689,org.apache.commons.math3.util.MathArrays#51,org.apache.commons.math3.util.MathArrays#88,org.apache.commons.math3.util.MathArrays#89,org.apache.commons.math3.util.MathArrays#92,org.apache.commons.math3.util.MathArrays#93,org.apache.commons.math3.util.MathArrays#94,org.apache.commons.math3.util.MathArrays#96,org.apache.commons.math3.util.MathArrays#110,org.apache.commons.math3.util.MathArrays#111,org.apache.commons.math3.util.MathArrays#114,org.apache.commons.math3.util.MathArrays#115,org.apache.commons.math3.util.MathArrays#116,org.apache.commons.math3.util.MathArrays#118,org.apache.commons.math3.util.MathArrays#132,org.apache.commons.math3.util.MathArrays#133,org.apache.commons.math3.util.MathArrays#136,org.apache.commons.math3.util.MathArrays#137,org.apache.commons.math3.util.MathArrays#138,org.apache.commons.math3.util.MathArrays#140,org.apache.commons.math3.util.MathArrays#154,org.apache.commons.math3.util.MathArrays#155,org.apache.commons.math3.util.MathArrays#158,org.apache.commons.math3.util.MathArrays#159,org.apache.commons.math3.util.MathArrays#160,org.apache.commons.math3.util.MathArrays#162,org.apache.commons.math3.util.MathArrays#173,org.apache.commons.math3.util.MathArrays#174,org.apache.commons.math3.util.MathArrays#175,org.apache.commons.math3.util.MathArrays#177,org.apache.commons.math3.util.MathArrays#188,org.apache.commons.math3.util.MathArrays#189,org.apache.commons.math3.util.MathArrays#190,...,org.apache.commons.math3.random.RandomDataImpl#127,org.apache.commons.math3.random.RandomDataImpl#128,org.apache.commons.math3.random.RandomDataImpl#136,org.apache.commons.math3.random.RandomDataImpl#156,org.apache.commons.math3.random.RandomDataImpl#161,org.apache.commons.math3.random.RandomDataImpl#166,org.apache.commons.math3.random.RandomDataImpl#186,org.apache.commons.math3.random.RandomDataImpl#191,org.apache.commons.math3.random.RandomDataImpl#196,org.apache.commons.math3.random.RandomDataImpl#213,org.apache.commons.math3.random.RandomDataImpl#218,org.apache.commons.math3.random.RandomDataImpl#233,org.apache.commons.math3.random.RandomDataImpl#248,org.apache.commons.math3.random.RandomDataImpl#265,org.apache.commons.math3.random.RandomDataImpl#279,org.apache.commons.math3.random.RandomDataImpl#293,org.apache.commons.math3.random.RandomDataImpl#307,org.apache.commons.math3.random.RandomDataImpl#320,org.apache.commons.math3.random.RandomDataImpl#336,org.apache.commons.math3.random.RandomDataImpl#363,org.apache.commons.math3.random.RandomDataImpl#383,org.apache.commons.math3.random.RandomDataImpl#401,org.apache.commons.math3.random.RandomDataImpl#415,org.apache.commons.math3.random.RandomDataImpl#431,org.apache.commons.math3.random.RandomDataImpl#447,org.apache.commons.math3.random.RandomDataImpl#461,org.apache.commons.math3.random.RandomDataImpl#462,org.apache.commons.math3.random.RandomDataImpl#472,org.apache.commons.math3.random.RandomDataImpl#473,org.apache.commons.math3.random.RandomDataImpl#485,org.apache.commons.math3.random.RandomDataImpl#486,org.apache.commons.math3.random.RandomDataImpl#493,org.apache.commons.math3.random.RandomDataImpl#494,org.apache.commons.math3.random.RandomDataImpl#518,org.apache.commons.math3.random.RandomDataImpl#519,org.apache.commons.math3.random.RandomDataImpl#532,org.apache.commons.math3.random.RandomDataImpl#550,org.apache.commons.math3.random.RandomDataImpl#565,org.apache.commons.math3.random.RandomDataImpl#581,Pass/Fail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PASS
1,0,0,0,0,0,0,0,0,

In [ ]:
x = df[spectra_list[:len(spectra_list)-1]]
y = df["Pass/Fail"]

In [ ]:
fails = []
passes = []
i = 0
for result in y:
    if result == "FAIL":
        fails.append(i)
    else:
        passes.append(i)
    i += 1 

In [ ]:
if len(fails) == 1:
    df = df.append(df.iloc[fails[0]])
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [ ]:
if (df.shape[0] < 6):
    df = df.append(df)
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
model = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt', random_state=1)

In [ ]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
exp_dalex = dalex.Explainer(model, data=x_train, y=y_train, model_type="classification")

In [ ]:
breakdown_fail = exp_dalex.predict_parts(x.iloc[18], type="break_down")
breakdown_fail_interaction = exp_dalex.predict_parts(x.iloc[18], type="break_down_interactions")

In [ ]:
dalex_lines = {}

for index, row in breakdown_fail.result.iterrows():
    if index == 0 or index == len(breakdown_fail.result) - 1:
        continue
    dalex_lines[row['variable_name']] = row['contribution']

In [ ]:
with open("Math-8-dalex.txt", "w") as file:
    for i in dalex_lines:
        file.write(str(i) + ";" + str(dalex_lines[i]) + "\n")